<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#first-version" data-toc-modified-id="first-version-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>first version</a></span></li><li><span><a href="#new-version" data-toc-modified-id="new-version-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>new version</a></span></li></ul></div>

## first version

In [18]:
### import things
import pandas as pd
import copy
### read form the csv doc
data_train = pd.read_csv('train.csv')  

data_test = pd.read_csv('test.csv')

### trans to array
train_array = np.array(data_train)
test_array = np.array(data_test)
###############hyper parameter
use_normA=0
use_normy=0
###############hyper parameter

### process the A to be norm or not
if use_normA==1: 
    A_before_norm = train_array
    A=np.empty(np.shape(A_before_norm))
    for i in range(np.shape(A_before_norm)[1]):
        temp=A_before_norm[:,i]/np.linalg.norm(A_before_norm[:,i])
        A[:,i]=temp
else:
    A = train_array

### the A shape
[d,n] = A.shape


### aha! same as the lambda, the L-1 norm coefficent
lamb=1e-4

### L-max
Lmax=(0.25*max(sum(A*A)))
Lmax=3*Lmax
### the learning rate 0.15/(the proxi-L constant)
# alpha=0.15/(0.25*max(sum(A*A)))

### tau1 
tau1=lamb/Lmax

### loops
iters_outer=2
iters_inner=2*d

### count the number bingo
right=0

xx_0=np.zeros([85*67,67])
for i in range(67):
    xx_0[i*85:i*85+85][:,i]=1

for k in range(100):
    rand_index=np.random.randint(85*67)
#     print('rand_index',rand_index)
    rand_who=int(rand_index/85)
#     print('rand_who',rand_who)
    
    #### which y version you use, norm processed or the plain one 
    if use_normy==1:
        temp_y = test_array[:,rand_index].reshape(d,1)
        y=np.empty(np.shape(temp_y))
        y=temp_y/np.linalg.norm(temp_y)
    else:
        y=test_array[:,rand_index].reshape(d,1)
    
    ### initialize the two ws
    w_tilda=np.array(np.zeros([n,1]))
    x_prev=np.array(np.zeros([n,1]))
    y_prev=np.array(np.zeros([n,1]))
    ### big loop
    for i in range(iters_outer):
        mu=(-A.T).dot(y-A.dot(w_tilda))/d
        tau=2/(i+4)
        tau=max(tau,0.1818)
        alpha=0.1/(tau*Lmax)
        tau2=lamb*alpha
        ##small loop
        for j in range(iters_inner):
            ### choose a rand one from d 
            idx=np.random.randint(0,d)
            
            ### updata w_prev
            w_prev=tau*x_prev+0.5*w_tilda+(0.5-tau)*y_prev
            
            ### A transpose, need reshape
            A_idx=A[idx].T.reshape(n,1)
            ### g_prev and tilda
            g_prev=(-A_idx).dot(y[idx]-(A[idx].reshape(1,n)).dot(w_prev)) 
            g_tilda=(-A_idx).dot(y[idx]-(A[idx].reshape(1,n)).dot(w_tilda)) 
            ### update x_prev
            g_svrg=g_prev-g_tilda+mu
            x_prev=x_prev-alpha*(g_svrg)
            
            ### update the parameter
#             w_prev=w_prev-alpha*(g_prev-g_tilda+mu)
            ### the soft thresholding
            temp=w_prev-tau1
            temp=(temp+abs(temp))/2
            w_prev=np.sign(w_prev)*temp
        ### assign the w_tilda for another loop  2 versions  
        w_tilda=copy.deepcopy(w_prev)
        


        
    res_min=1e9
    for i in range(np.shape(xx_0)[1]):
        temp=xx_0[:,i].reshape(5695,1)
        x_phi=w_tilda*temp
        res=sum((y-A.dot(x_phi))*(y-A.dot(x_phi)))
        if res<res_min:
            res_min=res
            index_min=i


    print('randwho and index_min',rand_who,index_min)
#     print('detect and rand_who',detect,rand_who)
#     print('rand_index',rand_index)
#     print('rand_who',rand_who)
#     print('*'*80)
#     if detect == rand_who:
#         right=right+1

    if rand_who == index_min:
        right=right+1


    
    
    

randwho and index_min 5 5
randwho and index_min 44 44
randwho and index_min 19 19
randwho and index_min 64 64
randwho and index_min 24 45
randwho and index_min 45 64
randwho and index_min 41 41
randwho and index_min 50 50
randwho and index_min 34 19
randwho and index_min 50 50
randwho and index_min 4 4
randwho and index_min 53 53
randwho and index_min 22 22
randwho and index_min 14 45
randwho and index_min 26 26
randwho and index_min 48 24
randwho and index_min 48 48
randwho and index_min 11 47
randwho and index_min 22 22
randwho and index_min 10 10
randwho and index_min 42 24
randwho and index_min 18 18
randwho and index_min 41 41
randwho and index_min 39 39
randwho and index_min 62 62
randwho and index_min 7 22
randwho and index_min 15 45
randwho and index_min 25 52
randwho and index_min 50 50
randwho and index_min 36 36
randwho and index_min 53 53
randwho and index_min 44 44
randwho and index_min 49 49
randwho and index_min 11 24
randwho and index_min 26 26
randwho and index_min 38 

## new version

In [7]:
### import things
import pandas as pd
import copy
import seaborn as sns
import time
### read form the csv doc
data_train = pd.read_csv('train.csv')  
data_test = pd.read_csv('test.csv')

### trans to array
train_array = np.array(data_train)
test_array = np.array(data_test)
###############hyper parameter
use_normA=0
use_normy=0
###############hyper parameter

###############functions

### determin the x_tilda
def cp(x):
    return copy.deepcopy(x)

### get w_tilda
def compute_w_tilda(iters_inner,w_prev,ww_prev,all_or_one='mix',alpha=0.8,beta=0.2):
    if all_or_one=='all':
        w_tilda=cp(ww_prev)/iters_inner
    elif all_or_one=='one':
        w_tilda=cp(w_prev)
    elif all_or_one=='mix':
        w_tilda= cp(alpha*(w_prev)+beta*(ww_prev/iters_inner))
    return w_tilda

### get the data matrix A
def get_A(use_normA,train_array):
    if use_normA==1: 
        A_before_norm = train_array
        A=np.empty(np.shape(A_before_norm))
        for i in range(np.shape(A_before_norm)[1]):
            temp=A_before_norm[:,i]/np.linalg.norm(A_before_norm[:,i])
            A[:,i]=temp
    else:
        A = train_array
    return A

### get alpha 
def compute_alpha(A,lamb):
    L_max=0.25*max(sum(A*A))
    alpha=0.15/(L_max+lamb)
    return alpha

def get_xx_0():
    xx_0=np.zeros([85*67,67])
    for i in range(67):
        xx_0[i*85:i*85+85][:,i]=1
    return xx_0

def compute_res(y,x_phi,lamb):
    return sum((y-A.dot(x_phi))*(y-A.dot(x_phi)))
    
def get_index_min(xx_0,w_tilda):
    res_min=1e9
    for m in range(np.shape(xx_0)[1]):
        temp=xx_0[:,m].reshape(5695,1)
        x_phi=w_tilda*temp
        res=compute_res(y,x_phi,lamb)
        if res<res_min:
            res_min=res
            index_min=m
    return index_min

def compute_sci(xx_0,w):
    w_l1=np.linalg.norm(w,ord=1)
    temp_max=1e-9
    for m in range(np.shape(xx_0)[1]):
        temp=xx_0[:,m].reshape(5695,1)
        x_phi=w_tilda*temp
        x_phi_l1=np.linalg.norm(x_phi,ord=1)
        if x_phi_l1>temp_max:
            temp_max=x_phi_l1
    return (67*(temp_max/w_l1)-1)/(67-1)



###############functions


### process the A to be norm or not
A=get_A(0,train_array)
[d,n] = A.shape



### aha! same as the lambda, the L-1 norm coefficent
lamb=1e-5

### L-max
Lmax=(0.25*max(sum(A*A)))
Lmax=3*Lmax
### the learning rate 0.15/(the proxi-L constant)
# alpha=0.15/(0.25*max(sum(A*A)))

### tau1 
tau1=lamb/Lmax

### loops
iters_outer=2
iters_inner=2*d

### make only part of the list to be none zero
xx_0=get_xx_0()


### count the number bingo
right=0



rand_index=108
rand_who=int(rand_index/85)

#### which y version you use, norm processed or the plain one 
if use_normy==1:
    temp_y = test_array[:,rand_index].reshape(d,1)
    y=np.empty(np.shape(temp_y))
    y=temp_y/np.linalg.norm(temp_y)
else:
    y=test_array[:,rand_index].reshape(d,1)

### initialize the two ws
w_tilda=np.array(np.zeros([n,1]))
x_prev=np.array(np.zeros([n,1]))
y_prev=np.array(np.zeros([n,1]))

### w_prev hotmap
w_prev_hot=np.empty([n,1])
hot_num=0



### store the res and time
res_list=[]
time_list=[]
sci_list=[]
### big loop

### count the time 
start = time.clock()
for i in range(iters_outer):
    mu=(-A.T).dot(y-A.dot(w_tilda))/d
    tau=2/(i+4)
    tau=max(tau,0.1818)
    alpha=0.1/(tau*Lmax)
    tau2=lamb*alpha
    ##small loop
    for j in range(iters_inner):
        ### choose a rand one from d 
        idx=np.random.randint(0,d)

        ### updata w_prev
        w_prev=tau*x_prev+0.5*w_tilda+(0.5-tau)*y_prev

        ### A transpose, need reshape
        A_idx=A[idx].T.reshape(n,1)
        ### g_prev and tilda
        g_prev=(-A_idx).dot(y[idx]-(A[idx].reshape(1,n)).dot(w_prev)) 
        g_tilda=(-A_idx).dot(y[idx]-(A[idx].reshape(1,n)).dot(w_tilda)) 
        ### update x_prev
        g_svrg=g_prev-g_tilda+mu
        x_prev=x_prev-alpha*(g_svrg)

        ### update the parameter
#             w_prev=w_prev-alpha*(g_prev-g_tilda+mu)
        ### the soft thresholding
        temp=w_prev-tau1
        temp=(temp+abs(temp))/2
        w_prev=np.sign(w_prev)*temp

        ### compute the res of w_prev and store
        res_now=sum((y-A.dot(w_prev))*(y-A.dot(w_prev)))
        res_list.append(res_now)
        time_list.append(time.clock()-start)
        
        ### compute the sci value of w_prev and store
        sci_now=compute_sci(xx_0,w_prev)
        sci_list.append(sci_now)
    
        ### make the w_prev_hot
        w_prev_hot=np.concatenate((w_prev_hot,w_prev),axis=1)
        
         
        
    ### assign the w_tilda for another loop  2 versions  
    w_tilda=copy.deepcopy(w_prev)


index_min=get_index_min(xx_0,w_tilda)


print(('rand_index',rand_index))
print(('randwho and index_min',rand_who,index_min))

if rand_who == index_min:
    right=right+1

time_list_name="npy/ka"+'time_list'+'inner'+str(iters_inner)+'outer'+str(iters_outer)+'.npy'
res_list_name="npy/ka"+'res_list'+'inner'+str(iters_inner)+'outer'+str(iters_outer)+'.npy'
sci_list_name="npy/ka"+'sci_list'+'inner'+str(iters_inner)+'outer'+str(iters_outer)+'.npy'
np.save(time_list_name,time_list)
np.save(res_list_name,res_list)
np.save(sci_list_name,sci_list)



('rand_index', 108)
('randwho and index_min', 1, 0)


[0.19734119528790472]